In [1]:
import json, re

## Aux functions

In [2]:
UF_IDS ={
    "Acre": 12,
    "Alagoas": 27,
    "Amazonas": 13,
    "Amapá": 16,
    "Bahia": 29,
    "Ceará": 23,
    "Espírito Santo": 32,
    "Goiás": 52,
    "Maranhão": 21,
    "Minas Gerais": 31,
    "Mato Grosso do Sul": 50,
    "Mato Grosso": 51,
    "Pará": 15,
    "Paraíba": 25,
    "Pernambuco": 26,
    "Piauí": 22,
    "Paraná": 41,
    "Rio de Janeiro": 33,
    "Rio Grande do Norte": 24,
    "Rondônia": 11,
    "Roraima": 14,
    "Rio Grande do Sul": 43,
    "Santa Catarina": 42,
    "Sergipe": 28,
    "São Paulo": 35,
    "Tocantins": 17,
    "Distrito Federal": 53
}

"""
    This function receives a GeoJSON data and updates the 'id' field of each feature
    with the corresponding ID of the UF_IDS dictionary.
    
    :param geojson_data: GeoJSON data
    :return: GeoJSON data with updated 'id' field
"""
def update_UF_id(geojson_data):
    for feature in geojson_data['features']:
        nm_uf = feature['properties'].get('NM_UF')
        if nm_uf in UF_IDS:
            feature['id'] = UF_IDS[nm_uf]
        else:
            print(f"NM_UF '{nm_uf}' not found in the ID dictionary.")

    return geojson_data

In [3]:
"""
    This function receives a GeoJSON data and removes the key informed from the properties of each feature.

    :param geojson_data: GeoJSON data
    :param key_to_remove: Key to be removed from the properties
    :return: GeoJSON data without the key informed
"""
def remove_keys_from_properties(geojson_data, key_to_remove):
    for feature in geojson_data['features']:
        if key_to_remove in feature['properties']:
            del feature['properties'][key_to_remove]

    return geojson_data

In [4]:
"""
    This function receives a GeoJSON data and renames the keys that match the old_key_pattern

    :param geojson_data: GeoJSON data
    :param old_key_pattern: Key pattern to be removed
    :param new_key_prefix: Prefix to be added to the key
    :return: GeoJSON data with the keys renamed
"""
def rename_keys(geojson_data, old_key_pattern, new_key_prefix):
    if isinstance(geojson_data, dict):
        keys_to_rename = []  
        
        # Identify the keys that need to be renamed
        for key in list(geojson_data.keys()):
            if re.match(old_key_pattern, key):
                new_key = re.sub(old_key_pattern, new_key_prefix, key)
                keys_to_rename.append((key, new_key))

        # Now rename the identified keys
        for old_key, new_key in keys_to_rename:
            geojson_data[new_key] = geojson_data.pop(old_key)  # Rename the key

        # Recursively call the function for values that are dictionaries or lists
        for key in geojson_data:
            if isinstance(geojson_data[key], (dict, list)):
                rename_keys(geojson_data[key], old_key_pattern, new_key_prefix)

    elif isinstance(geojson_data, list):
        for item in geojson_data:
            rename_keys(item, old_key_pattern, new_key_prefix)

    return geojson_data

In [5]:
"""
    This function receives a GeoJSON data and adds a new metadata field to each feature with the sum of the values of the keys that match the key_prefix.

    :param geojson_data: GeoJSON data
    :param key_prefix: Prefix of the keys to be considered
    :param metadata_name: Name of the new metadata field

    :return: GeoJSON data with the new metadata field
"""
def sum_metadata(geojson_data, key_prefix, metadata_name):
    for feature in geojson_data['features']:
        properties = feature['properties']
        values = [value for key, value in properties.items() if key.startswith(key_prefix)]
        
        if values:
            final_value = sum(values)
            properties[metadata_name] = final_value 

    return geojson_data

In [6]:
"""
    This function receives an area in Km² and returns a formatted string with the area in Km² or Mil Km².

    :param area: Area in Km²
    :return: Formatted area
"""
def format_area(area):
    if area > 1000:
        return f"{round(area / 1000, 1)} Mil Km²"
    else:
        return f"{round(area, 1)} Km²"

"""
    This function receives a municipalities number and returns a formatted string.

    :param municipalities: Number of municipalities
    :return: Formatted string
"""
def format_municipalities(municipalities):
    if municipalities == 1:
        return f"{municipalities} município"
    else:
        return f"{municipalities} municípios"

"""
    This function receives a density and returns a formatted string.

    :param density: Density
    :return: Formatted string
"""
def format_density_hab(density):
    return f"Média estadual: {density} hab/Km²"

"""
    This function receives the mortality and returns a formatted string.

    :param mortality: Mortality
    :return: Formatted string
"""
def format_mortality(mortality):
    return f"Média estadual: {mortality} óbitos por mil nascidos vivos"

"""
    This function receives the mean and returns a formatted string.

    :param mean: Mean
    :return: Formatted string
"""
def format_estadual(mean):
    return f"Média estadual: {mean}"

"""
    This function receives a GeoJSON data, a key prefix and a function to format the values of the keys that match the key prefix.

    :param geojson_data: GeoJSON data
    :param key_prefix: Prefix of the keys to be formatted
    :param format_func: Function to format the values of the keys

    :return: GeoJSON data with the formatted keys
"""
def format_key(geojson_data, key_prefix, format_func):
    for feature in geojson_data['features']:
        properties = feature['properties']
        for key in properties:
            if key.startswith(key_prefix) and isinstance(properties[key], (int, float)):
                properties[key] = format_func(properties[key])

    return geojson_data

## Climate and Enviromental data

In [7]:
inner_geojson_path = './2020.geojson'
original_geojson_path = './mapMetadata/brazil-states.geojson'

In [17]:
with open(inner_geojson_path, 'r') as f1:
    inner_geojson = json.load(f1)

    # Update UFs IDs
    inner_geojson = update_UF_id(inner_geojson)

    # Rename keys
    inner_geojson = remove_keys_from_properties(inner_geojson, "NM_UF")
    inner_geojson = remove_keys_from_properties(inner_geojson, "SIGLA_UF")

    # Rename keys
    inner_geojson = rename_keys(inner_geojson, r"^Area_km2_", "Area_info_")

    # Add metadata
    inner_geojson = sum_metadata(inner_geojson, "Area_info_", "Area_general")

    # Format keys values
    inner_geojson = format_key(inner_geojson, "Area_info", format_area)
    inner_geojson = format_key(inner_geojson, "Area_general", format_area)

    # Save changes to the original file
    with open(inner_geojson_path, 'w') as f2:
        json.dump(inner_geojson, f2, ensure_ascii=False, indent=4)


## Socio-economic data

In [17]:
inner_geojson_path = './FCs/Cisternas_Mun_500_FC.geojson'
original_geojson_path = './mapMetadata/brazil-states.geojson'

In [18]:
with open(inner_geojson_path, 'r') as f1:
    inner_geojson = json.load(f1)

    # Update UFs IDs
    inner_geojson = update_UF_id(inner_geojson)

    # Remove keys
    inner_geojson = remove_keys_from_properties(inner_geojson, "NM_UF")
    inner_geojson = remove_keys_from_properties(inner_geojson, "SIGLA_UF")
    # inner_geojson = remove_keys_from_properties(inner_geojson, "AREA_KM2")
    # inner_geojson = remove_keys_from_properties(inner_geojson, "CD_UF")
    # inner_geojson = remove_keys_from_properties(inner_geojson, "NM_REGIAO")

    # Rename keys
    inner_geojson = rename_keys(inner_geojson, r"^Range_", "Area_info_")
    # inner_geojson = rename_keys(inner_geojson, r"^Area_km2_", "Area_info_")
    # inner_geojson = rename_keys(inner_geojson, r"^Meta_info", "Area_general")

    # Format keys values
    inner_geojson = format_key(inner_geojson, "Area_info", format_municipalities)
    # inner_geojson = format_key(inner_geojson, "Area_general", format_estadual)

    # Save changes to the original file
    with open(inner_geojson_path, 'w') as f2:
        json.dump(inner_geojson, f2, ensure_ascii=False, indent=4)


## Save final result 

In [14]:
inner_geojson_path = './FCs/Cisternas_Mun_500_FC.geojson'
original_geojson_path = './mapMetadata/brazil-states.geojson'
original_geojson_index_key = 'analfgeneral'

In [15]:
"""
    This function receives a GeoJSON data and updates the 'id' field of each feature

    :param geojson_data: GeoJSON data
    :param index_key: Index key
    
    :return: GeoJSON data with updated 'id' field
"""
def merge_geojsons_properties_by_id(geojson_destination, geojson_source, new_object_name):
    # Create a dictionary to map the id and properties in the geojson_source
    source_map = {feature['id']: feature['properties'] for feature in geojson_source['features']}

    for feature in geojson_destination['features']:
        feature_id = feature['id']
        if feature_id in source_map:
            feature['properties'][new_object_name] = source_map[feature_id]

    return geojson_destination


In [16]:
with open(original_geojson_path, 'r') as f1:
    with open(inner_geojson_path, 'r') as f2:
        geojson_data = json.load(f1)
        inner_geojson = json.load(f2)
        geojson_destino_atualizado = merge_geojsons_properties_by_id(geojson_data, inner_geojson, original_geojson_index_key)         

        with open(original_geojson_path, 'w') as f3:
            json.dump(geojson_destino_atualizado, f3, ensure_ascii=False, indent=4)